In [50]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Normalizer,StandardScaler,LabelEncoder
import tensorflow
import keras

In [51]:
train = pd.read_csv('train_df.csv')
test = pd.read_csv('test_df.csv')

In [52]:
train.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,55 Cancri e,PSO J318.5-22,TRAPPIST-1e,Earth,Europa,Mars,CryoSleep,VIP
0,39.0,0.0,0.0,0.0,0.0,0.0,False,0,0,1,0,1,0,0,0
1,24.0,109.0,9.0,25.0,549.0,44.0,True,0,0,1,1,0,0,0,0
2,58.0,43.0,3576.0,0.0,6715.0,49.0,False,0,0,1,0,1,0,0,1
3,33.0,0.0,1283.0,371.0,3329.0,193.0,False,0,0,1,0,1,0,0,0
4,16.0,303.0,70.0,151.0,565.0,2.0,True,0,0,1,1,0,0,0,0


In [53]:
bins = [0, 20, 40, 60, 80]
labels = ['Teens', 'Youngs', 'Adult','Elderly']

train['Age'] = pd.cut(x = train['Age'], bins = bins, labels = labels, include_lowest = True)
test['Age'] = pd.cut(x = test['Age'], bins = bins, labels = labels, include_lowest = True)

In [54]:
train_age = pd.get_dummies(train['Age'])
test_age = pd.get_dummies(test['Age'])

In [55]:
train.drop('Age',axis=1,inplace=True)
test.drop('Age',axis=1,inplace=True)

In [56]:
train = pd.concat([train,train_age],axis=1)
test = pd.concat([test,test_age],axis=1)

In [57]:
print(train.shape)
print(test.shape)

(8693, 18)
(4277, 17)


In [58]:
test.head()

,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,55 Cancri e,PSO J318.5-22,TRAPPIST-1e,Earth,Europa,Mars,CryoSleep,VIP,Teens,Youngs,Adult,Elderly
0,0.0,0.0,0.0,0.0,0.0,0,0,1,1,0,0,1,0,0,1,0,0
1,0.0,9.0,0.0,2823.0,0.0,0,0,1,1,0,0,0,0,1,0,0,0
2,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,0,1,0,0,1,0,0
3,0.0,6652.0,0.0,181.0,585.0,0,0,1,0,1,0,0,0,0,1,0,0
4,10.0,0.0,635.0,0.0,0.0,0,0,1,1,0,0,0,0,1,0,0,0


In [59]:
norm = Normalizer()

col = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']

train[col] = norm.fit_transform(train[col])
test[col] = norm.fit_transform(test[col])

In [61]:
train.to_csv('preprocessed_train.csv',index=False)
test.to_csv('preprocessed_test.csv',index=False)

In [63]:
train.shape

(8693, 18)

In [107]:
model = keras.Sequential([

    keras.layers.Dense(500,activation='relu',input_dim=17),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(250,activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(100,activation='relu'),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(1,activation='sigmoid'),
])

model.compile(optimizer='adam',loss='binary_crossentropy',metrics='accuracy')

In [108]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_35 (Dense)            (None, 500)               9000      
                                                                 
 dropout_9 (Dropout)         (None, 500)               0         
                                                                 
 dense_36 (Dense)            (None, 250)               125250    
                                                                 
 dropout_10 (Dropout)        (None, 250)               0         
                                                                 
 dense_37 (Dense)            (None, 100)               25100     
                                                                 
 dropout_11 (Dropout)        (None, 100)               0         
                                                                 
 dense_38 (Dense)            (None, 1)               

In [109]:
X = train.drop('Transported',axis=1)
y = train['Transported']

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y = encoder.fit_transform(y)

train_x, test_x, train_y, test_y = train_test_split(X,y,test_size=0.2,random_state=1234,stratify=y)

In [110]:
model.fit(train_x,train_y,epochs=30,validation_split=0.2)

Epoch 1/30


2023-05-28 14:00:45.112982: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


174/174 [==============================] - ETA: 0s - loss: 0.5044 - accuracy: 0.7589

2023-05-28 14:00:47.465683: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


174/174 [==============================] - 3s 14ms/step - loss: 0.5044 - accuracy: 0.7589 - val_loss: 0.4440 - val_accuracy: 0.7815
Epoch 2/30
174/174 [==============================] - 2s 10ms/step - loss: 0.4657 - accuracy: 0.7811 - val_loss: 0.4496 - val_accuracy: 0.8023
Epoch 3/30
174/174 [==============================] - 2s 10ms/step - loss: 0.4543 - accuracy: 0.7884 - val_loss: 0.4330 - val_accuracy: 0.8001
Epoch 4/30
174/174 [==============================] - 2s 10ms/step - loss: 0.4508 - accuracy: 0.7965 - val_loss: 0.4299 - val_accuracy: 0.8023
Epoch 5/30
174/174 [==============================] - 2s 9ms/step - loss: 0.4466 - accuracy: 0.7976 - val_loss: 0.4397 - val_accuracy: 0.7951
Epoch 6/30
174/174 [==============================] - 2s 10ms/step - loss: 0.4469 - accuracy: 0.8001 - val_loss: 0.4379 - val_accuracy: 0.7901
Epoch 7/30
174/174 [==============================] - 2s 10ms/step - loss: 0.4445 - accuracy: 0.7972 - val_loss: 0.4285 - val_accuracy: 0.8001
Epoch 8/30


In [111]:
model.evaluate(test_x,test_y)

55/55 [==============================] - 0s 6ms/step - loss: 0.4538 - accuracy: 0.7769


[0.45377662777900696, 0.7768833041191101]

In [112]:
model.evaluate(train_x,train_y)

218/218 [==============================] - 1s 5ms/step - loss: 0.4155 - accuracy: 0.8105


[0.4154946208000183, 0.8104687929153442]

In [120]:
y_pred = model.predict(test)

134/134 [==============================] - 0s 2ms/step


In [121]:
y = []

for x in y_pred:
    if x >=0.5:
        y.append('True')
    else:
        y.append('False')

In [114]:
df = pd.read_csv('test.csv')

In [122]:
y = pd.DataFrame(y)

In [124]:
y.head()

,0
0,True
1,False
2,True
3,True
4,True


In [126]:
submission = pd.concat([df['PassengerId'],y],axis=1,keys=['PassengerId','Transported'])

In [127]:
submission.head()

,PassengerId,Transported
,PassengerId,0
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True


In [128]:
submission.to_csv('submission.csv',index=False)